In [8]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pylab as pl
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, classification_report
from sklearn.cluster import MiniBatchKMeans
from preprocess import *
from collections import Counter
import joblib
import cv2
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# for faster execution using gpu
#import cupy as cp
import numpy as np
import multiprocessing
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
with open('fonts-dataset/font_names.txt') as file:
    lines = file.readlines()
    styles = [line.rstrip()[4:] for line in lines] # ignoring number at first of each style (first 4 chars)
print(styles)

['ScheherazadeNew', 'marhey', 'lemonada', 'ibm']


In [3]:
x=[]
y=[]
for i in range(0, len(styles)):
    for filename in sorted(glob.glob('fonts-dataset/'+str(i)+'/*.jpeg')):
        img = cv2.imread(filename)
     #   resized_img = cv2.resize(img, (200, 200))
        x.append(img)
        y.append(i)



In [4]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)


In [50]:
x_train_preprocessed = []
i=0
y_train_modified=[]

for img in x_train:
    binr=threshold_image(img)
    binr=assure_white_bg(binr)
    img=denoise(img)
    words=get_words_grey(binr,img, True)
    words = np.array(words)
    if len(words) > 19:
        random_row_indices = np.random.choice(words.shape[0], size=19, replace=False)    
        words = words[random_row_indices]

    x_train_preprocessed.extend(words)
    y_train_modified.extend([y_train[i]]*len(words))
    i=i+1
print("x_train_preprocessed: ", len(x_train_preprocessed))

x_train_preprocessed:  57954


In [36]:
def lbp_calculated_pixel(img, x, y):
    center = img[x, y]
    top_left = img[x-1, y-1]
    top = img[x-1, y]
    top_right = img[x-1, y+1]
    right = img[x, y+1]
    bottom_right = img[x+1, y+1]
    bottom = img[x+1, y]
    bottom_left = img[x+1, y-1]
    left = img[x, y-1]
    # get lbp values
    binary_values = np.array([top_left, top, top_right, right, bottom_right, bottom, bottom_left, left]) >= center
    binary_values =  binary_values.reshape((8, -1)).T.astype(int)
    powers_of_two = 2 ** np.arange(8)
    powers_of_two = powers_of_two[::-1]
    values=binary_values * powers_of_two
    values=np.sum(values, axis=1)
    # normalize the values
    values=values/np.sum(values)
   # print(values)
    return values

def lbp_feature_vector(img):
    img=np.array(img)
    height, width = img.shape
    indices_x, indices_y = np.meshgrid(np.arange(1, height - 1), np.arange(1, width - 1), indexing='ij')
    feature_vector = lbp_calculated_pixel(img, indices_x, indices_y)
    return feature_vector.flatten()
    
def prepare_data(images):
    data = []
    for img in images:
        data.append(lbp_feature_vector(img))
    standardized_data = StandardScaler().fit_transform(data)
    pca = PCA(n_components=0.95)
    pca.fit(standardized_data)
    data= pca.transform(standardized_data)
    print(pca.n_components_)
    return data,pca

In [52]:
x_train_prepared,pca= prepare_data(x_train_preprocessed)


2665


In [53]:
x_train_prepared=np.array(x_train_prepared)
print(x_train_prepared.shape)

(57954, 2665)


In [54]:
print("x_train_preprocessed: ", len(x_train_prepared))
print("y_train_modified: ", len(y_train_modified))
from classifiers_manager import ClassifiersManager,Classifiers
classifiers_manager = ClassifiersManager()
classifiers_manager.train(Classifiers.random_forest,x_train_prepared,y_train_modified)

x_train_preprocessed:  57954
y_train_modified:  57954


In [ ]:
img_predicted_labels = np.array([])
for i in range(0,len(x_test)):
    binr=threshold_image(x_test[i])
    binr=assure_white_bg(binr)
    img=denoise(x_test[i])
    words=get_words_grey(binr,img, True)
    
    words = np.array(words)
    if len(words) > 19:
        random_row_indices = np.random.choice(words.shape[0], size=19, replace=False)    
        words = words[random_row_indices]
    
    if(len(words) > 0):
        data = []
        for img in words:
            data.append(lbp_feature_vector(img))
        standardized_data = StandardScaler().fit_transform(data)
        data= pca.transform(standardized_data)

        features= np.array(data)
    
        words_predicted_labels= classifiers_manager.predict(Classifiers.random_forest,features)
        preds_count= Counter(words_predicted_labels)
        most_common_label = preds_count.most_common(1)[0][0]
        img_predicted_labels = np.append(img_predicted_labels, most_common_label)
    else:    
        img_predicted_labels = np.append(img_predicted_labels, 0)

print(classifiers_manager.getClassificationReport(img_predicted_labels, y_test, styles))


                 precision    recall  f1-score   support

ScheherazadeNew       0.51      0.53      0.52       186
         marhey       0.63      0.35      0.45       192
       lemonada       0.41      0.77      0.53       205
            ibm       0.48      0.25      0.33       217

       accuracy                           0.47       800
      macro avg       0.50      0.47      0.46       800
   weighted avg       0.50      0.47      0.45       800



In [27]:
print(img_predicted_labels)
print("---------------")
print(y_test)


[2. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 1. 0. 3. 0. 0. 0. 0. 2. 0. 0. 2. 0. 1.
 3. 0. 0. 0. 2. 3. 0. 1. 0. 0. 2. 0. 0. 0. 1. 3. 0. 2. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 3. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 2. 2. 0. 0. 3. 0. 0. 0.
 0. 3. 0. 0. 0. 0. 3. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 3. 0. 3. 0. 1. 0. 0. 0. 2. 2. 1. 3. 0. 3. 0. 0. 0.
 0. 3. 0. 0. 0. 1. 0. 3. 2. 0. 1. 0. 0. 0. 0. 3. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 3. 3. 0. 0. 0. 0. 2. 0. 1. 0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 1. 2. 0. 0.
 2. 2. 2. 0. 0. 0. 2. 0. 0. 3. 0. 3. 0. 1. 0. 0. 0. 0. 1. 0. 0. 3. 0. 0.
 0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 3. 0. 0. 0. 2. 0. 1. 3. 3. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 2. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0.
 0. 1. 0. 3. 0. 1. 0. 1. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0.
 0. 0. 1. 0. 0. 0. 2. 0. 3. 0. 0. 3. 0. 0. 0. 1. 2. 0. 1. 0. 0. 0. 0. 3.
 2. 0. 0. 0. 1. 0. 0. 0. 0. 0. 3. 0. 2. 1. 0. 0. 0. 0. 1. 2. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 3. 0. 0. 2. 2. 0. 1. 2. 1.